In [1]:
import pandas as pd

artigo_treino = pd.read_csv("treino.csv")
artigo_teste = pd.read_csv("teste.csv")
artigo_treino.sample()

,title,text,date,category,subcategory,link
72446,"Em meio a perigos, equipes de resgate salvam r...","Como sempre, tudo começa com uma chamada de te...",2017-04-18,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...


## Count Vectorizer

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
texto = ["tenha um bom dia",
         "tenha um péssimo dia",
         "tenha um ótimo dia",
         "tenha um dia ruim"]

vetorizador = CountVectorizer()
vetorizador.fit(texto)

CountVectorizer()

In [4]:
print(vetorizador.vocabulary_)

{'tenha': 4, 'um': 5, 'bom': 0, 'dia': 1, 'péssimo': 2, 'ótimo': 6, 'ruim': 3}


In [5]:
vetor_bom = vetorizador.transform(["bom"])
print(vetor_bom.toarray())

[[1 0 0 0 0 0 0]]


In [6]:
vetor_otimo = vetorizador.transform(["ótimo"])
print(vetor_otimo.toarray())

[[0 0 0 0 0 0 1]]


## Word2Vec

In [7]:
from gensim.models import KeyedVectors

modelo = KeyedVectors.load_word2vec_format("cbow_s300.txt")

In [8]:
modelo.most_similar("china")

[('rússia', 0.7320704460144043),
 ('índia', 0.7241616249084473),
 ('tailândia', 0.701935887336731),
 ('indonésia', 0.6860769987106323),
 ('turquia', 0.6741335988044739),
 ('malásia', 0.6665689945220947),
 ('mongólia', 0.6593616008758545),
 ('manchúria', 0.6581847071647644),
 ('urss', 0.6581669449806213),
 ('grã-bretanha', 0.6568096876144409)]

In [9]:
modelo.most_similar("itália")

[('frança', 0.7983610033988953),
 ('grécia', 0.7547340989112854),
 ('espanha', 0.7417077422142029),
 ('holanda', 0.7343162298202515),
 ('grã-bretanha', 0.728174090385437),
 ('inglaterra', 0.7172061204910278),
 ('bélgica', 0.7155542373657227),
 ('alemanha', 0.7135661840438843),
 ('suécia', 0.6831520199775696),
 ('áustria', 0.6822685599327087)]

In [10]:
modelo.most_similar(positive=["brasil", "argentina"])

[('chile', 0.6781662702560425),
 ('peru', 0.634803295135498),
 ('venezuela', 0.6273865103721619),
 ('equador', 0.6037014126777649),
 ('bolívia', 0.6017141342163086),
 ('haiti', 0.5993806719779968),
 ('méxico', 0.596230685710907),
 ('paraguai', 0.5957703590393066),
 ('uruguai', 0.5903672575950623),
 ('japão', 0.5893509984016418)]

In [11]:
# nuvem -> nuvens : estrelas -> estrela
# nuvens + estrela - nuvem = estrelas

modelo.most_similar(positive=["nuvens", "estrela"], negative="nuvem")

[('estrelas', 0.5497430562973022),
 ('plêiades', 0.3791979253292084),
 ('colinas', 0.3746805489063263),
 ('trovoadas', 0.373703271150589),
 ('sombras', 0.3734194040298462),
 ('pombas', 0.3726758360862732),
 ('corredoras', 0.3640727698802948),
 ('cigarras', 0.36065393686294556),
 ('galáxias', 0.35754913091659546),
 ('luas', 0.3575345277786255)]

In [12]:
modelo.most_similar(positive=["médico", "mulher"], negative="homem")

[('enfermeira', 0.6180862188339233),
 ('psicóloga', 0.47447526454925537),
 ('dama-de-companhia', 0.47382354736328125),
 ('esposa', 0.46559134125709534),
 ('parteira', 0.46362870931625366),
 ('mãe', 0.45817679166793823),
 ('governanta', 0.45722925662994385),
 ('madrasta', 0.4569782614707947),
 ('menina', 0.44435909390449524),
 ('filha', 0.44341111183166504)]

In [13]:
modelo.most_similar(positive=["professor", "mulher"], negative="homem")

[('professora', 0.6192208528518677),
 ('aluna', 0.5449554324150085),
 ('esposa', 0.4978230595588684),
 ('ex-aluna', 0.4884248375892639),
 ('namorada', 0.4737858772277832),
 ('enfermeira', 0.4728144407272339),
 ('filha', 0.467373788356781),
 ('irmã', 0.4584590792655945),
 ('ex-namorada', 0.45824769139289856),
 ('ex-professora', 0.45104706287384033)]

## Vetorização de texto

In [14]:
artigo_treino.title.loc[12]

"Daniel Craig será stormtrooper em novo 'Star Wars', diz ator"

In [15]:
import nltk
import string
nltk.download("punkt")

def tokenizador(texto):
    texto = texto.lower()
    lista_alfanumerica = []

    for token_valido in nltk.word_tokenize(texto):
        if token_valido in string.punctuation: continue
        lista_alfanumerica.append(token_valido)

    return lista_alfanumerica

tokenizador("Texto Exemplo, 1234.")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\paulo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['texto', 'exemplo', '1234']

In [16]:
import numpy as np

# Combinação de vetores por soma
def combinacao_vetores(palavras_numeros, modelo):
    vetor_resultante = np.zeros(300)

    for pn in palavras_numeros:
        try:
            vetor_resultante += modelo.get_vector(pn)
        except KeyError:
            if pn.isnumeric():
                pn = "0" * len(pn) # "0, 00, 000, 0000, etc. dependendo do tamanho de caracteres do número
                vetor_resultante += modelo.get_vector(pn)
            else:
                vetor_resultante += modelo.get_vector("unknown")

    return vetor_resultante

In [17]:
palavras_numeros = tokenizador("texto caelumx")
vetor_texto = combinacao_vetores(palavras_numeros, modelo)
print(len(vetor_texto))
print(vetor_texto)

TypeError: combinacao_vetores() missing 1 required positional argument: 'modelo'

In [ ]:
def matriz_vetores(textos, modelo):
    x = len(textos)
    y = 300 # número de dimensões do dicionário/vocabulário cbow300
    matriz = np.zeros((x, y))

    for i in range(x):
        palavras_numeros = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_vetores(palavras_numeros, modelo)

    return matriz

matriz_vetores_treino = matriz_vetores(artigo_treino.title, modelo)
matriz_vetores_teste = matriz_vetores(artigo_teste.title, modelo)
print(matriz_vetores_treino.shape)
print(matriz_vetores_teste.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=200)
lr.fit(matriz_vetores_treino, artigo_treino.category)

In [ ]:
lr.score(matriz_vetores_teste, artigo_teste.category)

In [ ]:
artigo_teste.category.unique()

In [ ]:
from sklearn.metrics import classification_report

label_prevista = lr.predict(matriz_vetores_teste)
cr = classification_report(artigo_teste.category, label_prevista)
print(cr)

In [ ]:
from sklearn.dummy import DummyClassifier
dc = DummyClassifier()
dc.fit(matriz_vetores_treino, artigo_treino.category)
label_prevista_dc = dc.predict(matriz_vetores_teste)
cr_dummy = classification_report(artigo_teste.category, label_prevista_dc)
print(cr_dummy)

Skip-gram

In [ ]:
modelo_skipgram = KeyedVectors.load_word2vec_format("skip_s300.txt")

In [ ]:
matriz_vetores_treino_skipgram = matriz_vetores(artigo_treino.title, modelo_skipgram)
matriz_vetores_teste_skipgram = matriz_vetores(artigo_teste.title, modelo_skipgram)

In [ ]:
lr_skipgram = LogisticRegression(max_iter=300)
lr_skipgram.fit(matriz_vetores_treino_skipgram, artigo_treino.category)

In [ ]:
label_prevista_skigram = lr_skipgram.predict(matriz_vetores_teste_skipgram)
cr_skipgram = classification_report(artigo_teste.category, label_prevista_skigram)
print(cr)